In [1]:
import numpy as np
import pandas as pd
import sys, os
import glob
from pprint import pprint
import csv

In [2]:
import nibabel as nib
from nilearn import image, plotting

/home/sathiesh/.conda/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/sathiesh/.conda/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [3]:
from pathlib import Path

In [24]:
df_train_image_paths = pd.read_csv('/data-nas/brains/MICCAI_BraTS2018/interim/survival_data.csv')
df_train_image_paths.head()

,BraTS18ID,Age,Survival,ResectionStatus
0,Brats18_TCIA08_167_1,74.907,153,NaN
1,Brats18_TCIA08_242_1,66.479,147,NaN
2,Brats18_TCIA08_319_1,64.860,254,NaN
3,Brats18_TCIA08_469_1,63.899,519,NaN
4,Brats18_TCIA08_218_1,57.345,346,NaN


In [25]:
NB_DIR = %pwd
NB_DIR = Path(NB_DIR)
subtype = 'HGG' #change this if error 
image = 't1'

In [26]:
DATA = Path('/data-nas/brains/MICCAI_BraTS2018/interim/'+subtype)

In [27]:
studies_path = sorted(glob.glob(str(DATA)+'/*'))#/home/sathiesh/BRAT18/HGG/Brats18_2013_10_1

In [28]:
len(studies_path)

210

In [29]:
names_studies = [s.split("/")[-1] for s in studies_path] #Brats18_2013_10_1

In [30]:
names_studies[:2]

['Brats18_2013_10_1', 'Brats18_2013_11_1']

In [13]:
brains_fns = glob.glob(str(DATA) + '/*/*seg.nii.gz') #finner alle pathene som matcher stringen
brains_fns[:2]

['/data-nas/brains/MICCAI_BraTS2018/interim/HGG/Brats18_2013_11_1/Brats18_2013_11_1_seg.nii.gz',
 '/data-nas/brains/MICCAI_BraTS2018/interim/HGG/Brats18_2013_14_1/Brats18_2013_14_1_seg.nii.gz']

In [14]:
len(brains_fns)

210

In [18]:
brains = ["_".join(fn.split('/')[-1:]) for fn in brains_fns] #concat strings 
brains[:5] # 'CMU_b_0050648_brain.nii.gz'

['Brats18_2013_11_1_seg.nii.gz',
 'Brats18_2013_14_1_seg.nii.gz',
 'Brats18_2013_19_1_seg.nii.gz',
 'Brats18_2013_22_1_seg.nii.gz',
 'Brats18_2013_26_1_seg.nii.gz']

# Create CSV 

In [34]:
CSV_PATH = NB_DIR/'../../csv/brats/interim'
CSV_PATH.mkdir(exist_ok=True)

In [35]:
masks = []
#Brats18_CBICA_ABM_1_seg

In [59]:
def create_niftynet_csv(dataset, masks=masks, outputdir=CSV_PATH):
    """
    Input: 
        dataset: path to a folder containing studies with labelled data
        masks: a list of masks to use
    Output:
        - One CSV containing "ID, original image" for all studies
        - One CSV containing "ID, brain image" for all studies
        - Multiple CSVs, one for each label type (aseg, aparc+aseg, ribbon, etc)
    """
    
    # Create label CSVs
    
    for m in masks:
        masks_studies =[f"{f}/{m}" for f in dataset]

        labels = list(zip(names_studies, masks_studies))
     
        labelsfn = f'{outputdir}/{m.split(".")[0]}_labels_{subtype}.csv'
    
        with open(labelsfn,'w') as out:
            csv_out=csv.writer(out)
            #csv_out.writerow(['ID','location'])
            for row in labels:
                csv_out.writerow(row)
            print(f'{labelsfn} created')
            
    # Create data CSVs
    
    for fn in ['_seg.nii.gz', f'_{image}.nii.gz']:
        images_studies = [f"{f}/{f.split('/')[-1]}{fn}" for f in dataset]
        data = list(zip(names_studies, images_studies))
        
        datafn = f'{outputdir}/{fn.split(".")[0]}_{subtype}.csv'
        
        with open(datafn,'w') as out:
            csv_out=csv.writer(out)
            #csv_out.writerow(['ID','location'])
            for row in data:
                csv_out.writerow(row)
            print(f'{datafn} created')

In [60]:
create_niftynet_csv(studies_path)

/home/sathiesh/niftynet_brain/nbs/brats/../../csv/brats/interim/_seg_HGG.csv created
/home/sathiesh/niftynet_brain/nbs/brats/../../csv/brats/interim/_t1_HGG.csv created


# Concatenate HGG.csv and LGG.csv

In [50]:
#check if interim file contains hgg and lgg files

In [51]:
def concate_df(subtypes, new_filename): 
    df_arr = []
    for filename in subtypes: 
        df_arr.append(pd.read_csv(f'{CSV_PATH}/{filename}', header=None))
    
    df = pd.concat(df_arr, ignore_index=True)
    df.to_csv(f'{CSV_PATH}/../{new_filename}', index=False, header=None)

In [52]:
subtypes = [f'orig{image}_std_HGG.csv',f'orig{image}_std_LGG.csv']
new_filename = f'orig_{image}.csv'
concate_df(subtypes, new_filename)

In [59]:
subtypes = ['brain_HGG.csv','brain_LGG.csv']
new_filename = 'labels.csv'
concate_df(subtypes, new_filename)